### -------------Yanolja crawling---------------

In [54]:
import scrapy, requests
from scrapy.http import TextResponse
import pandas as pd
import json

In [2]:
# 1. 프로젝트 생성

In [14]:
!rm -rf yanolja_hotel
!scrapy startproject yanolja_hotel

New Scrapy project 'yanolja_hotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress/yanolja_hotel

You can start your first spider with:
    cd yanolja_hotel
    scrapy genspider example example.com


In [15]:
!tree yanolja_hotel

yanolja_hotel
├── scrapy.cfg
└── yanolja_hotel
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py
    └── spiders
        └── __init__.py

2 directories, 7 files


In [16]:
# 2. xpath 찾기 : headers 설정

In [2]:
import requests #호텔 key값 얻는 모듈 생성 -> yanolja.py
import datetime

def get_hotel_keys():    
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36", 
           "X-Requested-With": "XMLHttpRequest"
              }
    today = datetime.date.today()
    checkin_date = today.strftime("%Y-%m-%d")
    checkout_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    region = "서울"
    num=0
    page = 1
    key_info = []
    while True:
        parent_url = f"https://www.yanolja.com/api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0\
    &checkinDate={checkin_date}&checkoutDate={checkout_date}&hotel=1&keyword=서울&lat=37.50681&lng=127.06624&page={page}&limit=20&\
    searchKeyword={region}&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes="
        req = requests.get(parent_url, headers=headers)
        response = req.json()
        #print('page=', str(apage))
        page += 1

        if response["motels"]["counts"] > 0:
            key = response["motels"]
            key_info.append(key)
        else:
            break
    hotel_keys = []
    for i in key_info:
        for j in i['lists']:
            hotel_keys.append(j['key'])        
    return hotel_keys

In [3]:
hotel_keys = get_hotel_keys()
len(hotel_keys)

416

### ------------xpath 확인 -> df확인 --------------

In [4]:
links = []
url = "https://www.yanolja.com/hotel/"
for hotel_key in hotel_keys:
    hotel_url = url + hotel_key
    links.append(hotel_url)
len(links)

416

In [5]:
# link 확인
link = links[0]
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://www.yanolja.com/hotel/3013410>

In [11]:
# xpath 확인
import json
location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
    room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
else:    
    room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
    room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
else:
    room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
price = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[4]/div/div[5]/span/text()').extract()
prices=[]
for num in range(len(room_type)):
    price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
    price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
    prices.append(price)
        
location, name, level, score, review_count, room_type, room_detail, prices

('서울특별시 영등포구 경인로 870',
 '페어필드 바이 메리어트 서울',
 '4성급',
 '4.7',
 '8,797',
 ['Premier Twin',
  'Premier King',
  'Superior Single',
  'Premier Twin',
  'Premier Twin',
  'Premier King',
  'Premier King',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier King'],
 ['특가야놀자☆After 21pm Check-in/연박불가',
  '특가야놀자☆After 21pm Check-in/연박불가',
  '룸온리',
  '특가야놀자☆선착순 고층 룸UP+27시간스테이',
  '특가야놀자☆선착순 고층 룸UP',
  '특가야놀자☆선착순 고층 룸UP+25시간',
  '특가야놀자☆선착순 고층 룸UP+27시간',
  '특가야놀자☆선착순 고층 룸UP+25시간',
  '특가야놀자☆27시간스테이',
  '특가야놀자☆27시간스테이',
  '특가야놀자☆선착순 고층 룸UP',
  '특가야놀자☆Room and Parking Package',
  '특가야놀자☆Room and Parking Package',
  'stay for Breakfast',
  'stay for Breakfast'],
 ['59,900원',
  '59,900원',
  '60,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '71,800원',
  '71,800원',
  '77,300원',
  '77,300원'])

In [20]:
# df 확인
locations, names, levels, scores, review_counts, room_types, room_details, prices_1, loading_urls = [], [], [], [], [] ,[], [], [], []

num1 = 0
for link in links[:10]:
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        req = requests.get(link, headers=headers)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")

        location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
        name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
        try:
            price = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[4]/div/div[5]/span/text()').extract()
            prices=[]
            for num in range(len(room_type)):
                price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
                price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
                prices.append(price)
        except:
            prices= ["예약마감"]
    except:
        location = "페이지 없음"
        location = "페이지 없음"
        name = "페이지 없음"
        level = "페이지 없음"
        score = "페이지 없음"
        review_count = "페이지 없음"
        room_type = ["페이지 없음"]
        room_detail = ["페이지 없음"]
        price = ["페이지 없음"]
        prices= ["페이지 없음"]
        
    location = [location] * len(room_type)
    name = [name] * len(room_type)
    level = [level] * len(room_type)
    score = [score] * len(room_type)
    review_count = [review_count] * len(room_type)
    link = [link] * len(room_type)
    
    locations.extend(location)
    names.extend(name)
    levels.extend(level)
    scores.extend(score)
    review_counts.extend(review_count)
    room_types.extend(room_type)
    room_details.extend(room_detail)
    prices_1.extend(prices)
    loading_urls.extend(link)
    num1+=1
    print(str(num1) ,name[0])

1 페어필드 바이 메리어트 서울
2 나인트리 프리미어 명동2
3 영등포 GMS 호텔
4 신라스테이 구로
5 노보텔 앰배서더 서울 용산
6 글래드 마포
7 신라스테이 마포
8 호텔 더 디자이너스 여의도
9 홀리데이인 익스프레스 서울 홍대
10 더 파크호텔 서울


In [21]:
df = pd.DataFrame({
    'Name': names, 'Level': levels, 'Score':scores, 'Location': locations,
    'RoomType': room_types,'Roomdetail': room_details ,'Price': prices_1, 'Link': loading_urls, 'Review' : review_counts,
})
df.tail()

,Name,Level,Score,Location,RoomType,Roomdetail,Price,Link,Review
203,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,파크 홈 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공","150,000원",https://www.yanolja.com/hotel/3016365,"1,478"
204,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,프리미어 패밀리 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공","150,000원",https://www.yanolja.com/hotel/3016365,"1,478"
205,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,파크 로얄 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공","270,000원",https://www.yanolja.com/hotel/3016365,"1,478"
206,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,패밀리 디럭스 트리플+욕조,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공",0원,https://www.yanolja.com/hotel/3016365,"1,478"
207,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,파크 듀얼 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공",0원,https://www.yanolja.com/hotel/3016365,"1,478"


### --------확인 끝 spider.py로 확장---------

In [85]:
%%writefile yanolja_hotel/yanolja_hotel/items.py
import scrapy

class YanoljaHotelItem(scrapy.Item):
    location = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review_count = scrapy.Field()
    link = scrapy.Field()
    room_type = scrapy.Field()
    price = scrapy.Field()
    flatform = scrapy.Field()

Overwriting yanolja_hotel/yanolja_hotel/items.py


In [47]:
# 3. spider.py

In [86]:
%%writefile yanolja_hotel/yanolja_hotel/spiders/spider.py
import scrapy
from yanolja_hotel.items import YanoljaHotelItem
from yanolja import get_hotel_keys
import json
import datetime

class Spider(scrapy.Spider):
    name = "YanoljaHotel"
    allow_domain = ["https://www.yanolja.com"]
    start_urls = ["https://www.yanolja.com/hotel/"]
    
    def parse(self, response):
        hotel_keys = get_hotel_keys()
        start_urls = "https://www.yanolja.com/hotel/"
        links = []
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        for hotel_key in hotel_keys:
            hotel_url = start_urls + hotel_key
            links.append(hotel_url)
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
    
    def parse_content(self, response):
        item = YanoljaHotelItem()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
        
        roomtypes = list(zip(room_types, room_detail))

        prices=[]
        for num in range(len(room_types)):
            price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
            price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
            prices.append(price)

        locations = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        locations = json.loads(locations)["props"]["pageProps"]["placeInfo"]["address"]
        names = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        levels = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        scores = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_counts = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        links = response.url
        flatforms = response.url.split(".")[1]
        
        for i in range(len(roomtypes)):
            
            item['flatform'] = flatforms
            item['location'] = locations
            item['name'] = names
            item['level'] = levels
            item['score'] = scores
            item['review_count'] = review_counts
            item['link'] = links
            item['room_type'] = roomtypes[i]
            item['price'] = prices[i]
            
            yield item

Overwriting yanolja_hotel/yanolja_hotel/spiders/spider.py


In [65]:
# 4. settings.py : robots.txt 여부 파악 

In [66]:
# 5. 프로젝트 실행

In [78]:
%%writefile run.sh
cd yanolja_hotel
rm yanolja_hotel.csv
scrapy crawl YanoljaHotel -o yanolja_hotel.csv

Overwriting run.sh


In [82]:
!/bin/bash run.sh

2021-03-10 21:33:10 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: yanolja_hotel)
2021-03-10 21:33:10 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jan 15 2021, 05:57:06) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-10 21:33:10 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-10 21:33:10 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'yanolja_hotel',
 'NEWSPIDER_MODULE': 'yanolja_hotel.spiders',
 'SPIDER_MODULES': ['yanolja_hotel.spiders']}
2021-03-10 21:33:10 [scrapy.extensions.telnet] INFO: Telnet Password: d4887758badb3b2b
2021-03-10 21:33:10 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy

2021-03-10 21:33:15 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-10&checkoutDate=2021-03-11&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=12&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-10 21:33:15 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.yanolja.com:443
2021-03-10 21:33:15 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-10&checkoutDate=2021-03-11&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=13&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-10 21:33:15 [urllib3.connectionpool] DEBUG: Sta

2021-03-10 21:33:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001791>
{'flatform': 'yanolja',
 'level': '비즈니스',
 'link': 'https://www.yanolja.com/hotel/3001791',
 'location': '서울특별시 중구 장충단로13길 20',
 'name': '호텔 스카이파크 킹스타운 동대문점',
 'price_1': '63,300원',
 'review_count': '1,641',
 'room_type': ('주차불가,스탠다드 더블', '특가야놀자☆/9시 얼리체크인, 취소불가'),
 'score': '4.7'}
2021-03-10 21:33:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3012015>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3012015',
 'location': '서울특별시 강남구 테헤란로 610',
 'name': '글래드 강남 코엑스센터',
 'price_1': '0원',
 'review_count': '1,542',
 'room_type': ('스탠다드 트윈', '18시 체크인'),
 'score': '4.7'}
2021-03-10 21:33:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3012015>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3012015',
 'location': '서울특별시 강남구 테헤란로 610',
 'name': '글래드 강남 코엑스센터',

2021-03-10 21:33:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/1000102261>
{'flatform': 'yanolja',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/1000102261',
 'location': '서울특별시 종로구 인사동길 49',
 'name': '나인트리 프리미어 호텔 인사동',
 'price_1': '66,000원',
 'review_count': '1,463',
 'room_type': ('스탠다드 트윈', '야놀자단독ㅣ연박전용 Green Earth by Nine Tree'),
 'score': '4.8'}
2021-03-10 21:33:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3011846>
{'flatform': 'yanolja',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3011846',
 'location': '서울특별시 강남구 테헤란로 415',
 'name': 'L7 강남 바이 롯데',
 'price_1': '91,800원',
 'review_count': '1,492',
 'room_type': ('스탠다드 룸', '체크인시 배정'),
 'score': '4.8'}
2021-03-10 21:33:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3000643>
{'flatform': 'yanolja',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3000643',
 'location': '서울특별시 중구 마른내로 71',
 'name': '호텔 

2021-03-10 21:33:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001075>
{'flatform': 'yanolja',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3001075',
 'location': '서울특별시 영등포구 의사당대로 18',
 'name': '글래드 여의도',
 'price_1': '70,000원',
 'review_count': '1,789',
 'room_type': ('디럭스 트윈', '기준 2명 / 최대 2명'),
 'score': '4.7'}
2021-03-10 21:33:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001558>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001558',
 'location': '서울특별시 마포구 마포대로 109',
 'name': '롯데시티호텔 마포',
 'price_1': '0원',
 'review_count': '1,321',
 'room_type': ('디럭스 트윈', 'YA호특가★'),
 'score': '4.8'}
2021-03-10 21:33:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001558>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001558',
 'location': '서울특별시 마포구 마포대로 109',
 'name': '롯데시티호텔 마포',
 'price_1': '0원',
 'review_count'

...

2021-03-10 21:33:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001415>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001415',
 'location': '서울특별시 마포구 마포대로 83',
 'name': '신라스테이 마포',
 'price_1': '55,000원',
 'review_count': '3,163',
 'room_type': ('MY OFFICE', '숙박불가, 오후 이용권'),
 'score': '4.6'}
2021-03-10 21:33:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001415>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001415',
 'location': '서울특별시 마포구 마포대로 83',
 'name': '신라스테이 마포',
 'price_1': '60,000원',
 'review_count': '3,163',
 'room_type': ('체크인시 객실 배정', '숙박불가, 내맘대로 12시간STAY'),
 'score': '4.6'}
2021-03-10 21:33:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001415>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001415',
 'location': '서울특별시 마포구 마포대로 83',
 'name': '신라스테이 마포',
 'price_1': '6

2021-03-10 21:33:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001396>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001396',
 'location': '서울특별시 서대문구 충정로 76',
 'name': '신라스테이 서대문',
 'price_1': '60,500원',
 'review_count': '2,082',
 'room_type': ('내맘대로 12시간STAY', '숙박불가, 8-24시, 체크인시 객실 배정'),
 'score': '4.5'}
2021-03-10 21:33:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001415>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001415',
 'location': '서울특별시 마포구 마포대로 83',
 'name': '신라스테이 마포',
 'price_1': '0원',
 'review_count': '3,163',
 'room_type': ('MY OFFICE', '숙박불가, 오전 이용권'),
 'score': '4.6'}
2021-03-10 21:33:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001415>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001415',
 'location': '서울특별시 마포구 마포대로 83',
 'name': '신라스테이 마포',
 'price_1':

2021-03-10 21:33:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001396>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001396',
 'location': '서울특별시 서대문구 충정로 76',
 'name': '신라스테이 서대문',
 'price_1': '89,540원',
 'review_count': '2,082',
 'room_type': ('31시간 스테이', '스탠다드 더블, 3-16층'),
 'score': '4.5'}
2021-03-10 21:33:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001396>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001396',
 'location': '서울특별시 서대문구 충정로 76',
 'name': '신라스테이 서대문',
 'price_1': '91,960원',
 'review_count': '2,082',
 'room_type': ('내맘대로 인앤아웃 24 STAY PKG', '디럭스 더블, 17-27층'),
 'score': '4.5'}
2021-03-10 21:33:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001396>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001396',
 'location': '서울특별시 서대문구 충정로 76',
 'name': '신라스테이 서대문',
 'pr

2021-03-10 21:33:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001564',
 'location': '서울특별시 동작구 시흥대로 596',
 'name': '신라스테이 구로',
 'price_1': '60,500원',
 'review_count': '4,113',
 'room_type': ('스탠다드 트윈', '스트리밍 패키지'),
 'score': '4.6'}
2021-03-10 21:33:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009497>
{'flatform': 'yanolja',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3009497',
 'location': '서울특별시 중구 마른내로 28',
 'name': '나인트리 프리미어 명동2',
 'price_1': '115,000원',
 'review_count': '2,189',
 'room_type': ('스탠다드 트윈', '또또1+1ㅣ나인트리 전 지점 숙박권 1매, 주차불가'),
 'score': '4.8'}
2021-03-10 21:33:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009497>
{'flatform': 'yanolja',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3009497',
 'location': '서울특별시 중구 마른내로 28',
 'name': '나인트리 프리미어 명동2',
 'pr

2021-03-10 21:33:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001564',
 'location': '서울특별시 동작구 시흥대로 596',
 'name': '신라스테이 구로',
 'price_1': '90,750원',
 'review_count': '4,113',
 'room_type': ('스탠다드 트윈', '서프라이즈 패키지-석식'),
 'score': '4.6'}
2021-03-10 21:33:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001564',
 'location': '서울특별시 동작구 시흥대로 596',
 'name': '신라스테이 구로',
 'price_1': '90,750원',
 'review_count': '4,113',
 'room_type': ('더블', '31시간 스테이+TO GO 모닝도시락 2인, 15시체크인/22시체크아웃'),
 'score': '4.6'}
2021-03-10 21:33:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001564',
 'location': '서울특별시 동작구 시흥대로 596',
 'name': '신라스테이 구로',
 

In [83]:
data = pd.read_csv('./yanolja_hotel/yanolja_hotel.csv')

In [84]:
data

,flatform,level,link,location,name,price_1,review_count,room_type,score
0,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"59,900원","8,801","Premier Twin,특가야놀자☆After 21pm Check-in/연박불가",4.7
1,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"59,900원","8,801","Premier King,특가야놀자☆After 21pm Check-in/연박불가",4.7
2,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"60,000원","8,801","Superior Single,룸온리",4.7
3,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"63,000원","8,801","Premier Twin,특가야놀자☆선착순 고층 룸UP+27시간스테이",4.7
4,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"63,000원","8,801","Premier Twin,특가야놀자☆선착순 고층 룸UP",4.7
5,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"63,000원","8,801","Premier King,특가야놀자☆선착순 고층 룸UP+25시간",4.7
6,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"63,000원","8,801","Premier King,특가야놀자☆선착순 고층 룸UP+27시간",4.7
7,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"63,000원","8,801","Premier Twin,특가야놀자☆선착순 고층 룸UP+25시간",4.7
8,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"63,000원","8,801","Premier King,특가야놀자☆27시간스테이",4.7
9,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"63,000원","8,801","Premier Twin,특가야놀자☆27시간스테이",4.7


In [ ]:
# 6. pipeline.py : 크롤링한 데이터를 mysql에 저장

# -----------TEST------------